# Download iEEG

## Install libraries

In [1]:
%%capture
# Install ieegpy toolbox directly from github
# !pip install git+https://github.com/ieeg-portal/ieegpy.git
from ieeg.auth import Session
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matlab.engine
# Python Imports
import math
import re
import os
# Scipy Imports
from scipy import interpolate, stats, io
# My imports
from ieeg_utils import *
from signal_processing import *
from plotting import *
import time

## Load data

In [2]:
# Load HUP_implant_dates.xlsx
patients_df = pd.read_excel("./data/HUP_implant_dates.xlsx")
patients_df.head()

,ptID,IEEG_Portal_Number,Implant_Date,implant_time,Explant_Date,weight_kg
0,225,HUP225_phaseII,2021-10-18,07:15:00,2021-10-26 17:30:00,58.5
1,224,HUP224_phaseII,2021-10-13,07:15:00,2021-10-20 00:00:00,85.5
2,223,HUP223_phaseII,2021-09-29,07:15:00,2021-10-08 08:21:00,101.4
3,221,HUP221_phaseII,2021-08-16,07:15:00,2021-08-23 00:00:00,124.3
4,219,HUP219_phaseII,2021-07-12,07:15:00,2021-07-16 08:18:00,101.6


In [3]:
# Load aed_list.csv
aed_df = pd.read_csv("./data/aed_list.csv")
# Lowercase AED
aed_df["AED"] = aed_df["AED"].str.lower()
aed_df

,AED,Generic,Abbreviation
0,acetazolamide,acetazolamide,ACZ
1,ativan,lorazepam,LZP
2,banzel,rufinamide,RUF
3,brivaracetam,brivaracetam,BRI
4,carbamazepine,carbamazepine,CBZ
...,...,...,...
81,zarontin,ethosuximide,ETX
82,zonegran,zonisamide,ZNS
83,zonisamide,zonisamide,ZNS
84,aptiom,eslicarbazepine,xxx


In [4]:
# Load aed_ref_ranges.xlsx from ./data/
aed_ref_ranges_df = pd.read_excel("./data/aed_ref_ranges.xlsx")
# Lowercase Drug column
aed_ref_ranges_df["Drug"] = aed_ref_ranges_df["Drug"].str.lower()
# show unique units
print(aed_ref_ranges_df["Unit"].unique())
# mg/L and ug/mL are the same
# If Unit is ng/mL, convert to ug/mL
aed_ref_ranges_df.loc[aed_ref_ranges_df["Unit"] == "ng/mL", "Min"] = (
    aed_ref_ranges_df["Min"] / 1000
)
aed_ref_ranges_df.loc[aed_ref_ranges_df["Unit"] == "ng/mL", "Max"] = (
    aed_ref_ranges_df["Max"] / 1000
)
# Add a column that takes the average of Min and Max
aed_ref_ranges_df["Avg"] = (aed_ref_ranges_df["Min"] + aed_ref_ranges_df["Max"]) / 2
aed_ref_ranges_df

['mg/L' 'ug/mL' 'ng/mL']


,Drug,Min,Max,Unit,Avg
0,levetiracetam,12.00,46.00,mg/L,29.000
1,carbamazepine,4.00,10.00,mg/L,7.000
2,oxcarbazepine,3.00,35.00,ug/mL,19.000
3,clobazam,0.03,0.30,ng/mL,0.165
4,n-desmethylclobazam,0.30,3.00,ng/mL,1.650
5,topiramate,5.00,20.00,mg/L,12.500
6,valproic acid,50.00,125.00,ug/mL,87.500
7,lacosamide,1.00,10.00,ug/mL,5.500
8,felbamate,30.00,60.00,ug/mL,45.000
9,lamotrigine,2.50,15.00,mg/L,8.750


In [5]:
patient_ids = patients_df.ptID.to_numpy()
patient_weights = patients_df.weight_kg.to_numpy()
assert len(patient_ids) == len(patient_weights) == 80
len(patient_ids)

80

In [6]:
# Create a mapping between patient ids and the index of the patient in the patients_df dataframe.
patient_id_to_index = {}
for i, patient_id in enumerate(patient_ids):
    patient_id_to_index[patient_id] = i

In [7]:
ieeg_offset_row1_df = pd.read_excel("data/ieeg_offset/row_1.xlsx", header=None)
ieeg_offset_row2_df = pd.read_excel("data/ieeg_offset/row_2.xlsx", header=None)
ieeg_offset_row3_df = pd.read_excel("data/ieeg_offset/row_3.xlsx", header=None)

In [8]:
# Load rid_hup_table.csv from ./data/
rid_hup_table_df = pd.read_csv("./data/rid_hup_table.csv")
# Drop the t3_subject_id and ieegportalsubjno columns
rid_hup_table_df = rid_hup_table_df.drop(columns=["t3_subject_id", "ieegportalsubjno"])
rid_hup_table_df

,record_id,hupsubjno
0,623,35
1,624,36
2,625,37
3,626,38
4,627,39
...,...,...
212,534,250
213,923,251
214,918,252
215,864,253


In [9]:
# Load master_elecs.csv from ./data/
master_elecs_df = pd.read_csv("./data/master_elecs.csv")

# only take the numbers in rid column
master_elecs_df["rid"] = master_elecs_df["rid"].str.extract("(\d+)", expand=False)
master_elecs_df["rid"] = master_elecs_df["rid"].astype(int)

master_elecs_df

,rid,name,mni_x,mni_y,mni_z,mm_x,mm_y,mm_z,vox_x,vox_y,vox_z,label,soz,resected,spike_rate,engel
0,13,LST01,-34.416,-30.306,14.106,-34.6751,-0.42270,-39.4603,80.6116,106.5480,64.5941,left inferior temporal,False,False,1.091902,1.0
1,13,LST02,-40.867,-27.839,13.436,-43.0088,2.37777,-40.9321,72.0779,109.4150,63.1223,left inferior temporal,False,False,1.091902,1.0
2,13,LST03,-46.339,-24.613,16.392,-50.0126,5.26941,-35.3090,64.9060,112.3760,68.7455,EmptyLabel,False,False,1.419472,1.0
3,13,LST04,-46.326,-21.682,21.294,-49.9003,7.50003,-25.8205,65.0210,114.6600,78.2339,left middle temporal,False,False,0.655141,1.0
4,13,MST01,4.300,-67.361,16.011,15.2559,-41.60630,-33.6340,131.7410,64.3756,70.4205,right lingual,True,False,3.439490,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14212,785,RB08,55.240,-24.519,-12.397,54.7706,-22.78030,-19.5040,154.2550,114.2730,136.7560,EmptyLabel,False,NaN,0.369914,1.0
14213,785,RB09,60.019,-26.497,-11.675,58.9395,-25.23490,-19.5579,159.1350,111.9920,136.6960,EmptyLabel,False,NaN,0.665845,1.0
14214,785,RB10,65.500,-28.008,-9.952,63.7847,-27.57940,-18.5976,164.7520,109.9030,137.7640,right middle temporal,False,NaN,4.586930,1.0
14215,785,RB11,70.279,-29.985,-9.230,67.9536,-30.03410,-18.6515,169.6320,107.6220,137.7040,right middle temporal,False,NaN,2.071517,1.0


There are 80 patients.

### Helper functions for iEEG

In [10]:
def create_pwd_file(username, password, fname=None):
    if fname is None:
        fname = "{}_ieeglogin.bin".format(username[:3])
    with open(fname, "wb") as f:
        f.write(password.encode())
    print("-- -- IEEG password file saved -- --")


create_pwd_file("dma", "mycqEv-pevfo4-roqfan")

-- -- IEEG password file saved -- --


In [11]:
with open("dma_ieeglogin.bin", "r") as f:
    session = Session("dma", f.read())

## Download iEEG Data

In [12]:
eng = matlab.engine.start_matlab()
nina_path = eng.genpath("/Volumes/USERS/nghosn3/Pioneer/spikes-AED/aed_dose_modeling")
devin_path = eng.genpath("/Volumes/USERS/devinma/eeg-AED/matlab")
eng.addpath(nina_path, nargout=0)
eng.addpath(devin_path, nargout=0)

In [13]:
good_ids = np.array(
    [224, 223, 204, 202, 196, 190, 188, 187, 180, 178, 177, 174, 170, 169]
)
good_idx = np.array([patient_id_to_index[id] for id in good_ids])
good_idx

array([ 1,  2, 17, 18, 22, 28, 30, 31, 37, 39, 40, 42, 46, 47])

In [16]:
patients_processed = []
sample_rates = np.zeros_like(good_idx)

for i, patient_idx in enumerate(good_idx[:1]):
    # Get patient id and weight
    patient_id, patient_weight = (
        patient_ids[patient_idx],
        patient_weights[patient_idx],
    )

    #########################################
    # Check tables
    #########################################

    if rid_hup_table_df.loc[rid_hup_table_df["hupsubjno"] == patient_id].empty:
        print(f"HUP{patient_id} has no info on rid_hup_table sheet")
        continue

    patient_rid = int(
        rid_hup_table_df.loc[rid_hup_table_df["hupsubjno"] == patient_id].record_id
    )

    print(f"HUP{patient_id} has rid {patient_rid}")

    patient_electrodes_df = master_elecs_df.loc[master_elecs_df["rid"] == patient_rid]

    if patient_electrodes_df.empty:
        print(f"HUP{patient_id} has no info on master electrode sheet")
        continue

    #########################################
    # Get ieeg dataset name
    #########################################

    ieeg_dataset_names = []
    # Check if ieeg_offset_row1_df[patient_idx] is all NaNs
    if ieeg_offset_row1_df[patient_idx].isnull().values.all():
        ieeg_dataset_names = [f"HUP{patient_id}_phaseII"]
    else:
        ieeg_dataset_names = ieeg_offset_row1_df[patient_idx].dropna().to_list()

    print(f"HUP{patient_id} has {str(ieeg_dataset_names)} datasets")

    if len(ieeg_dataset_names) > 1:
        continue

    #########################################
    # Get the useful channels
    #########################################

    grey_matter_seeg_electrodes_indices = get_electrode_indicies(
        ieeg_dataset_names, session, patient_electrodes_df
    )

    #########################################
    # Get AED dose
    #########################################

    # Run Matlab function
    get_aed_curve_kg_res = eng.get_aed_curve_kg(
        np.array([patient_id]), np.array([patient_weight]), nargout=4
    )
    assert len(get_aed_curve_kg_res) == 4, "get_aed_curve_kg_res should have 4 outputs"

    # Assign results to variables
    all_dose_curves = get_aed_curve_kg_res[0][0]
    all_tHr = get_aed_curve_kg_res[1][0]
    all_med_names = get_aed_curve_kg_res[3][0]
    assert (
        len(all_dose_curves) == len(all_tHr) == len(all_med_names)
    ), "all_dose_curves, all_tHr, and all_med_names should have the same length"

    # Process numpy arrays
    all_med_names_plot = np.array(all_med_names).flatten()

    tmp = []
    for dose in all_dose_curves:
        dose = np.array(dose).flatten()
        tmp.append(dose)
    all_dose_curves_plot = np.array(tmp, dtype=object)

    tmp = []
    for dose in all_tHr:
        dose = np.array(dose).flatten()
        tmp.append(dose)
    all_tHr_plot = np.array(tmp, dtype=object)

    # Get the last element of each array in all_tHr_plot and test if they're the same
    # They should be the same because they're the same patient
    assert all(
        [all_tHr_plot[i][-1] == all_tHr_plot[0][-1] for i in range(len(all_tHr_plot))]
    ), "The last element of each array in all_tHr_plot should be the same"

    # Construct the time axis
    emu_start_time_hrs = min([all_tHr_plot[i][0] for i in range(len(all_tHr_plot))])
    emu_end_time_hrs = all_tHr_plot[0][-1]
    max_dose_duration_hrs = emu_end_time_hrs - emu_start_time_hrs

    #########################################
    # Get iEEG data
    #########################################

    for dataset_idx, dataset_name in enumerate(ieeg_dataset_names):
        eeg_offset_sec = ieeg_offset_row2_df[patient_idx][dataset_idx]
        eeg_offset_usec, eeg_offset_hrs = eeg_offset_sec * 1e6, int(
            eeg_offset_sec / 3600
        )
        assert (
            eeg_offset_sec > 0
        ), "ieeg_offset_sec should be positive, i.e. iEEG starts after AEDs"

        print(f"Opening {dataset_name}, it has offset of {eeg_offset_sec} seconds")
        # check if aligned_emu_start_time_hrs is an integer

        hourly_eeg = []
        eeg_end_time_hrs = None

        print(
            "Select electrodes in the grey matter"
            + str(grey_matter_seeg_electrodes_indices)
        )

        for hour in range(int(max_dose_duration_hrs) + 1):
            print(f"Getting iEEG data for hour {hour}")
            duration_usec = 1.8e8  # 3 minutes
            start_time_usec = hour * 3600 * 1e6
            stop_time_usec = start_time_usec + duration_usec

            local_res, sample_rate = get_iEEG_data(
                "dma",
                "dma_ieeglogin.bin",
                dataset_name,
                start_time_usec,
                stop_time_usec,
                select_electrodes=grey_matter_seeg_electrodes_indices,
            )

            assert local_res.shape[1] == len(grey_matter_seeg_electrodes_indices)

            sample_rates[i] = sample_rate
            print(sample_rate)
            print(sample_rates)

            # Drop rows that are all NaN
            local_res = local_res.dropna(axis=0, how="any")
            # Check if local_res is empty
            if local_res.empty:
                eeg_end_time_hrs = hour
                break
            else:
                # Save local_res as an excel file
                print(local_res.shape)
                np.save(f"./data/ieeg/HUP_{patient_id}_hour_{hour}.npy", local_res)

# Save sample rates to numpy file
np.save("./data/ieeg/good_ids.npy", good_ids)
np.save("./data/ieeg/sample_rates.npy", sample_rates)

HUP224 has rid 617
HUP224 has ['HUP224_phaseII'] datasets
HUP224
HUP224 has medications with NaN dose: 
Opening HUP224_phaseII, it has offset of 135178.0 seconds
Select electrodes in the grey matter[  7  15  19  20  27  28  35  37  38  39  42  43  44  45  46  47  54  55
  56  57  62  63  73  75  76  77  78  79  88  89  90  91  92  98 100 101
 105 106 107 108 109 110 112 119 120 122 124 126 127 130 131 137 138 141
 142 143 150 160 161 165 168 169 170 178 179 180 189 190 191 192 194 203
 204 214 219 222 224 226 237]
Getting iEEG data for hour 0
1024.0
[1024    0    0    0    0    0    0    0    0    0    0    0    0    0]
(182027, 79)
Getting iEEG data for hour 1
1024.0
[1024    0    0    0    0    0    0    0    0    0    0    0    0    0]
(184320, 79)
Getting iEEG data for hour 2
1024.0
[1024    0    0    0    0    0    0    0    0    0    0    0    0    0]
(184320, 79)
Getting iEEG data for hour 3
1024.0
[1024    0    0    0    0    0    0    0    0    0    0    0    0    0]
(184320, 

In [23]:
for hour in range(96):
    print(hour)
    x = np.load(f"./data/ieeg/HUP_224_hour_{hour}.npy")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45


KeyboardInterrupt: 